# Feature Engineering Time Series Analysis - Notepad-1

# Key Learning
  

1. Data Loading  
2. Other than Date column , use others as Predictor Variable

In [66]:
import pandas as pd
import matplotlib.pyplot as plt

import numpy as np
import seaborn as sns
#import klib
import dtale
import dtale.app as dtale_app
dtale_app.USE_COLAB = True

In [ ]:
! pip install dtale

In [33]:
data=pd.read_csv("/content/AirQualityUCI.csv",sep=";",parse_dates=[['Date', 'Time']])

<ipython-input-33-385d125c7900>:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data=pd.read_csv("/content/AirQualityUCI.csv",sep=";",parse_dates=[['Date', 'Time']])


In [34]:
data=data.iloc[:, :-2]

In [35]:
data.dropna(inplace=True)

In [36]:
data.shape


(9357, 14)

In [37]:
data.head()

,Date_Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,10/03/2004 18.00.00,"2,6",1360.0,150.0,"11,9",1046.0,166.0,1056.0,113.0,1692.0,1268.0,"13,6","48,9","0,7578"
1,10/03/2004 19.00.00,2,1292.0,112.0,"9,4",955.0,103.0,1174.0,92.0,1559.0,972.0,"13,3","47,7","0,7255"
2,10/03/2004 20.00.00,"2,2",1402.0,88.0,"9,0",939.0,131.0,1140.0,114.0,1555.0,1074.0,"11,9","54,0","0,7502"
3,10/03/2004 21.00.00,"2,2",1376.0,80.0,"9,2",948.0,172.0,1092.0,122.0,1584.0,1203.0,"11,0","60,0","0,7867"
4,10/03/2004 22.00.00,"1,6",1272.0,51.0,"6,5",836.0,131.0,1205.0,116.0,1490.0,1110.0,"11,2","59,6","0,7888"


In [38]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9357 entries, 0 to 9356
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date_Time      9357 non-null   object 
 1   CO(GT)         9357 non-null   object 
 2   PT08.S1(CO)    9357 non-null   float64
 3   NMHC(GT)       9357 non-null   float64
 4   C6H6(GT)       9357 non-null   object 
 5   PT08.S2(NMHC)  9357 non-null   float64
 6   NOx(GT)        9357 non-null   float64
 7   PT08.S3(NOx)   9357 non-null   float64
 8   NO2(GT)        9357 non-null   float64
 9   PT08.S4(NO2)   9357 non-null   float64
 10  PT08.S5(O3)    9357 non-null   float64
 11  T              9357 non-null   object 
 12  RH             9357 non-null   object 
 13  AH             9357 non-null   object 
dtypes: float64(8), object(6)
memory usage: 1.1+ MB


In [ ]:
data.describe()

In [39]:
new_var_names = [
    'Date_Time',
    'CO_true',
    'CO_sensor',
    'NMHC_true',
    'C6H6_true',
    'NMHC_sensor',
    'NOX_true',
    'NOX_sensor',
    'NO2_true',
    'NO2_sensor',
    'O3_sensor',
    'T',
    'RH',
    'AH',
]

In [40]:
data.columns = new_var_names

data.columns

Index(['Date_Time', 'CO_true', 'CO_sensor', 'NMHC_true', 'C6H6_true',
       'NMHC_sensor', 'NOX_true', 'NOX_sensor', 'NO2_true', 'NO2_sensor',
       'O3_sensor', 'T', 'RH', 'AH'],
      dtype='object')

In [43]:
predictors = data.columns[1:]

In [44]:
predictors

Index(['CO_true', 'CO_sensor', 'NMHC_true', 'C6H6_true', 'NMHC_sensor',
       'NOX_true', 'NOX_sensor', 'NO2_true', 'NO2_sensor', 'O3_sensor', 'T',
       'RH', 'AH'],
      dtype='object')

In [47]:
for var in predictors:
  if data[var].dtype=='O':
    data[var]=data[var].str.replace(',', '.')
    data[var]=pd.to_numeric(data[var])

In [49]:
data.head(5)

,Date_Time,CO_true,CO_sensor,NMHC_true,C6H6_true,NMHC_sensor,NOX_true,NOX_sensor,NO2_true,NO2_sensor,O3_sensor,T,RH,AH
0,10/03/2004 18.00.00,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,13.6,48.9,0.7578
1,10/03/2004 19.00.00,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255
2,10/03/2004 20.00.00,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502
3,10/03/2004 21.00.00,2.2,1376.0,80.0,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,11.0,60.0,0.7867
4,10/03/2004 22.00.00,1.6,1272.0,51.0,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,11.2,59.6,0.7888


In [52]:
data['Date_Time'] = data['Date_Time'].str.replace('.', ':', regex=False)

#data['Date_Time'] = pd.to_datetime(data['Date_Time'])

In [56]:
data['Date_Time'] = pd.to_datetime(data['Date_Time'], dayfirst=True)

In [57]:
data.sort_index(inplace=True)

In [58]:
data.head(5)

,Date_Time,CO_true,CO_sensor,NMHC_true,C6H6_true,NMHC_sensor,NOX_true,NOX_sensor,NO2_true,NO2_sensor,O3_sensor,T,RH,AH
0,2004-03-10 18:00:00,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,13.6,48.9,0.7578
1,2004-03-10 19:00:00,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255
2,2004-03-10 20:00:00,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502
3,2004-03-10 21:00:00,2.2,1376.0,80.0,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,11.0,60.0,0.7867
4,2004-03-10 22:00:00,1.6,1272.0,51.0,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,11.2,59.6,0.7888


In [60]:
data['Date_Time'].isna().sum()

0

In [61]:
data['Date_Time'].duplicated().sum()

0

In [62]:
data['Date_Time'].agg(['min', 'max'])

min   2004-03-10 18:00:00
max   2005-04-04 14:00:00
Name: Date_Time, dtype: datetime64[ns]

In [63]:
data.to_csv('/content/AirQualityUCI_ready.csv', index=False)

In [67]:
dtale.show(data)

https://fisayh9ishv-496ff2e9c6d22116-40000-colab.googleusercontent.com/dtale/main/1

In [74]:
data_copy=data

In [76]:
data_copy.head(5)

,CO_true,CO_sensor,NMHC_true,C6H6_true,NMHC_sensor,NOX_true,NOX_sensor,NO2_true,NO2_sensor,O3_sensor,T,RH,AH
Date_Time,,,,,,,,,,,,,
2004-03-10 18:00:00,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,13.6,48.9,0.7578
2004-03-10 19:00:00,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255
2004-03-10 20:00:00,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502
2004-03-10 21:00:00,2.2,1376.0,80.0,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,11.0,60.0,0.7867
2004-03-10 22:00:00,1.6,1272.0,51.0,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,11.2,59.6,0.7888


In [75]:
data_copy.set_index("Date_Time",inplace=True)

In [79]:
data_copy["Month"]=data_copy.index.month

In [80]:
data_copy["Week"]=data_copy.index.isocalendar().week

In [81]:
data_copy["Day"]=data_copy.index.day

In [84]:
data_copy["Day_Of_Week"]=data_copy.index.day_of_week

In [85]:
data_copy["Hour"]=data_copy.index.hour

In [88]:
data_copy["is_weekend"]=np.where(data_copy["Day_Of_Week"]>4,1,0)

In [89]:
dtale.show(data_copy)

https://fisayh9ishv-496ff2e9c6d22116-40000-colab.googleusercontent.com/dtale/main/2